Please note that this notebook is intended to be run in Google Colab rather than as a Jupyter notebook on your local machine. Please click the "Open in Colab" button.

# Setup

In [1]:
import os
import sys

# !wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
# !chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
# !bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
# !conda install -q -y -c conda-forge rdkit==2020.09.2

url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.7/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

!git clone https://github.com/chemprop/chemprop
os.chdir('chemprop')
!pip install -e .

sys.path.append('/usr/local/lib/python3.7/site-packages/')

import chemprop
import pandas as pd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3721    0  3721    0     0  13680      0 --:--:-- --:--:-- --:--:-- 13680
100 20.2M  100 20.2M    0     0  2339k      0  0:00:08  0:00:08 --:--:-- 2649k
Cloning into 'chemprop'...
remote: Enumerating objects: 15577, done.
remote: Total 15577 (delta 0), reused 0 (delta 0), pack-reused 15577
Receiving objects: 100% (15577/15577), 360.25 MiB | 26.11 MiB/s, done.
Resolving deltas: 100% (7892/7892), done.
Obtaining file:///content/chemprop
     |████████████████████████████████| 1.6 MB 5.3 MB/s 
     |████████████████████████████████| 3.1 MB 41.4 MB/s 
     |████████████████████████████████| 125 kB 69.9 MB/s 
     |████████████████████████████████| 199 kB 56.8 MB/s 
     |████████████████████████████████| 90 kB 8.2 MB/s 
     |████████████████████████████████| 100 kB 9.1 MB/s 
     |████████████████████████████████| 121 kB 48.0 MB

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/rdkit/Chem/PandasTools.py", line 131, in <module>
    if 'display.width' in pd.core.config._registered_options:
AttributeError: module 'pandas.core' has no attribute 'config'


# Train model

In [2]:
arguments = [
    '--data_path', 'tests/data/regression.csv',
    '--dataset_type', 'regression',
    '--save_dir', 'test_checkpoints',
    '--epochs', '10'
]

args = chemprop.args.TrainArgs().parse_args(arguments)
mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)

Command line
python /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-36ce3a9c-c294-4f10-a9c8-88931f83cd9d.json
Args
{'activation': 'ReLU',
 'adding_h': False,
 'aggregation': 'mean',
 'aggregation_norm': 100,
 'atom_descriptor_scaling': True,
 'atom_descriptors': None,
 'atom_descriptors_path': None,
 'atom_descriptors_size': 0,
 'atom_features_size': 0,
 'atom_messages': False,
 'batch_size': 50,
 'bias': False,
 'bias_solvent': False,
 'bond_feature_scaling': True,
 'bond_features_path': None,
 'bond_features_size': 0,
 'cache_cutoff': 10000,
 'checkpoint_dir': None,
 'checkpoint_frzn': None,
 'checkpoint_path': None,
 'checkpoint_paths': None,
 'class_balance': False,
 'config_path': None,
 'crossval_index_dir': None,
 'crossval_index_file': None,
 'crossval_index_sets': None,
 'cuda': False,
 'data_path': 'tests/data/regression.csv',
 'data_weights_path': None,
 'dataset_type': 'regression',
 'depth': 3,
 'depth_solvent': 3,


# Predict from file

In [ ]:
arguments = [
    '--test_path', 'tests/data/regression.csv',
    '--preds_path', 'test_preds.csv',
    '--checkpoint_dir', 'test_checkpoints'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

Loading training args
Setting molecule featurization parameters to default.
Loading data


500it [00:00, 112075.25it/s]
100%|██████████| 500/500 [00:00<00:00, 74520.36it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating SMILES
Test size = 500
Predicting with an ensemble of 1 models


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".



100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

Saving predictions to test_preds.csv
Elapsed time = 0:00:02


In [ ]:
df = pd.read_csv('tests/data/regression.csv')
df['preds'] = [x[0] for x in preds]
df

,smiles,logSolubility,preds
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.770,-1.169089
1,Cc1occc1C(=O)Nc2ccccc2,-3.300,-2.950731
2,CC(C)=CCCC(C)=CC(=O),-2.060,-1.805328
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.870,-7.098153
4,c1ccsc1,-1.330,0.095896
...,...,...,...
495,Nc1cc(nc(N)n1=O)N2CCCCC2,-1.989,-2.916466
496,Nc2cccc3nc1ccccc1cc23,-4.220,-4.451346
497,c1ccc2cc3c4cccc5cccc(c3cc2c1)c45,-8.490,-7.776431
498,OC(c1ccc(Cl)cc1)(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,-5.666,-5.773000


# Predict from SMILES list

In [ ]:
smiles = [['CCC'], ['CCCC'], ['OCC']]
arguments = [
    '--test_path', '/dev/null',
    '--preds_path', '/dev/null',
    '--checkpoint_dir', 'test_checkpoints'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args, smiles=smiles)

Loading training args


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Setting molecule featurization parameters to default.
Loading data
Validating SMILES
Test size = 3
Predicting with an ensemble of 1 models


  0%|          | 0/1 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".



100%|██████████| 1/1 [00:00<00:00,  1.36it/s]

Saving predictions to /dev/null
Elapsed time = 0:00:01


# Load model once, predict multiple times

In [ ]:
arguments = [
    '--test_path', '/dev/null',
    '--preds_path', '/dev/null',
    '--checkpoint_dir', 'test_checkpoints'
]

args = chemprop.args.PredictArgs().parse_args(arguments)

model_objects = chemprop.train.load_model(args=args)

smiles = [['CCC'], ['CCCC'], ['OCC']]
preds = chemprop.train.make_predictions(args=args, smiles=smiles, model_objects=model_objects)

smiles = [['CCCC'], ['CCCCC'], ['COCC']]
preds = chemprop.train.make_predictions(args=args, smiles=smiles, model_objects=model_objects)

Loading training args


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Setting molecule featurization parameters to default.
Loading data
Validating SMILES
Test size = 3
Predicting with an ensemble of 1 models


100%|██████████| 1/1 [00:00<00:00,  2.11it/s]


Saving predictions to /dev/null
Elapsed time = 0:00:00
Setting molecule featurization parameters to default.
Loading data
Validating SMILES
Test size = 3
Predicting with an ensemble of 1 models


100%|██████████| 1/1 [00:00<00:00,  1.78it/s]

Saving predictions to /dev/null
Elapsed time = 0:00:01
